### We will estimate the three-factor model using 5 years (2014-2018) of monthly returns on Facebook

In [6]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf

In [1]:
# download and unzip the data
# http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip


In [7]:
# define the parameters

RISKY_ASSET = 'FB'
START_DATE = '2013-12-31'
END_DATE = '2018-12-31'

In [28]:
# read the data and keep only the monthly data
from os import getcwd
from os.path import join

factor_df = pd.read_csv(join(getcwd(), 'F-F_Research_Data_Factors.csv'), skiprows=3) #


In [29]:
factor_df

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.38,-2.73,0.22
1,192608,2.64,-1.47,4.14,0.25
2,192609,0.36,-1.39,0.12,0.23
3,192610,-3.24,-0.13,0.65,0.32
4,192611,2.53,-0.16,-0.38,0.31
...,...,...,...,...,...
1233,2017,21.51,-4.95,-13.48,0.80
1234,2018,-6.93,-3.34,-9.80,1.81
1235,2019,28.28,-6.06,-10.48,2.14
1236,2020,23.67,12.77,-46.32,0.44


In [30]:
STR_TO_MATCH = ' Annual Factors: January-December '
indices = factor_df.iloc[:, 0] == STR_TO_MATCH
start_of_annual = factor_df[indices].index[0]
factor_df = factor_df[factor_df.index < start_of_annual]

In [32]:
# rename the columns, set a datetime index and filter by dates
factor_df.columns = ['date', 'mkt', 'smb', 'hml', 'rf']
factor_df['date'] = pd.to_datetime(factor_df['date'], format = '%Y%m').dt.strftime("%Y-%m")
factor_df = factor_df.set_index('date')
factor_df = factor_df.loc[START_DATE:END_DATE]

<ipython-input-32-30930474cf2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factor_df['date'] = pd.to_datetime(factor_df['date'], format = '%Y%m').dt.strftime("%Y-%m")


In [33]:
# convert the values to numeric values and divide by 100
factor_df = factor_df.apply(pd.to_numeric, errors = 'coerce').div(100)

In [34]:
factor_df.head()

,mkt,smb,hml,rf
date,,,,
2014-01,-0.0332,0.0092,-0.0202,0.0
2014-02,0.0465,0.0037,-0.0031,0.0
2014-03,0.0043,-0.0187,0.0492,0.0
2014-04,-0.0019,-0.0420,0.0114,0.0
2014-05,0.0206,-0.0189,-0.0013,0.0


In [35]:
# download fb prices

asset_df = yf.download(RISKY_ASSET, start = START_DATE, end = END_DATE,
                      adjusted = True)

[*********************100%***********************]  1 of 1 completed


In [40]:
# calculate monthly returns on the risky asset
y = asset_df['Adj Close'].resample('M').last().pct_change().dropna()
y.index = y.index.strftime('%Y-%m')
y.name = 'rtn'

In [42]:
# merge two datasets and calculate the excess returns
ff_data = factor_df.join(y)

In [44]:
ff_data['excess_rtn'] = ff_data.rtn - ff_data.rf

In [45]:
# estimate the 3 factor model
ff_model = smf.ols(formula = 'excess_rtn ~ mkt + smb + hml', data = ff_data).fit()
print(ff_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     5.065
Date:                Mon, 20 Sep 2021   Prob (F-statistic):            0.00357
Time:                        19:57:06   Log-Likelihood:                 88.253
No. Observations:                  60   AIC:                            -168.5
Df Residuals:                      56   BIC:                            -160.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0105      0.008      1.366      0.1

In [46]:
# we can also use pandas_datareader to get the datasets
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web


In [48]:
# print avalilable datasets
get_available_datasets()[:5]

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily']

In [49]:
# download the selected dataset 
ff_dict = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start = '2014-01-01')
ff_dict.keys()

dict_keys([0, 1, 'DESCR'])

In [50]:
ff_dict.get('DESCR')

'F-F Research Data Factors\n-------------------------\n\nThis file was created by CMPT_ME_BEME_RETS using the 202107 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc. Copyright 2021 Kenneth R. French\n\n  0 : (91 rows x 4 cols)\n  1 : Annual Factors: January-December (7 rows x 4 cols)'

In [51]:
# view the monthly dataset 
ff_dict[0].head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2014-01,-3.32,0.92,-2.02,0.0
2014-02,4.65,0.37,-0.31,0.0
2014-03,0.43,-1.87,4.92,0.0
2014-04,-0.19,-4.20,1.14,0.0
2014-05,2.06,-1.89,-0.13,0.0
